**STOCK PRICE PREDICTION USING LSTM**

Let's start by installing required packages and import them and the relevant data for stock prices

In [1]:
!pip install yfinance
!pip instal numpy
!pip install tensorflow
!pip install matplotlib
!pip install datetime

import numpy as np
import yfinance as yf
import tensorflow as tf
import os
from matplotlib import pyplot as plt
from datetime import datetime
import pandas as pd
from tensorflow.keras.layers import LSTM,Dense,SimpleRNN,Dropout,Add,Concatenate,Flatten
from tensorflow.keras import Model,Input


ERROR: unknown command "instal" - maybe you meant "install"
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 7.2 MB/s eta 0:00:00


You can fill the name of the option as you wish in the next line of code

In [3]:
info_stock = yf.Ticker("SASA.IS")
data_stock = info_stock.history(period="max")
data_stock = data_stock.sort_values('Date',ascending=True)

In [5]:
print(type(data_stock))
data_stock.head()

Stock price data is apparent since 2000 on a daily basis. Obviously, average prices of stocks are not provided but open and end prices and lowest and highest prices during the day are available. In addition, days when the dividend payments are made are provided as well as stock splits showing the changes in the issued number of stocks

In [6]:
print(f"Total Number of Dividend Payments: {sum(data_stock['Dividends'].values > 0)}")
print(f"Total Number of Stock Splits: {sum(data_stock['Stock Splits'].values > 0)}")

The changes in the stock prices in Dividend Days or Stock Split Days might be expected to be drastic. Let's see that

In [7]:
dividend_days = [index for index,value in enumerate(data_stock['Dividends'].values) if value > 0]
dividend_selected_days = sorted(np.concatenate([np.array(dividend_days) - 1,np.array(dividend_days),np.array(dividend_days) + 1],axis=0))
data_stock.iloc[np.array(dividend_selected_days),:]

On these two days, it seems that dividend payments were made. There didn't seem to occur any drops/jumps in the prices

In [ ]:
split_days = [index for index,value in enumerate(data_stock['Stock Splits'].values) if value > 0]
split_selected_days = sorted(np.concatenate([np.array(split_days[-2:]) - 1,np.array(split_days[-2:]),np.array(split_days[-2:]) + 1],axis=0))
data_stock.iloc[np.array(split_selected_days),:]

On these 2 days, it seems like there occured a stock split which lead to a drastic stock price decrease on 23rd but didn't seem to change the prices on the other split.



In [ ]:
def plot_function(seriesx,seriesy,labelx,labely,title):

  for i in range(len(seriesx)):
    if len(seriesx) != 1:
      plt.subplot(1,2,i+1)
    plt.plot(seriesx[i],seriesy[i])
    plt.xlabel(labelx)
    plt.ylabel(labely)
    plt.title(title)
    plt.xticks(rotation=50)

  plt.show()

plt.figure(figsize=(14,5))
plot_function([data_stock.index,data_stock.index[-500:]],[data_stock.Close,data_stock.Close[-500:]],"Date","Stock Price","Date vs. Stock Price")
plt.show()


Let's convert the model to a numpy array and then to tensorflow dataset to transorm the data into window format.

Let's set the batch size to a small number as 2 which is pretty much inline with SGD. In the last experiment, the batch size was quite high making the iterations more stable and faster but the model seemed to get stuck at a local minimum so lowering the batch size might help minimizing loss.

The values should be standardized as min max scaling the values might diminish training pace.

Instead of passing open, close, high etc. as they are, the current value will be passed as the ratio of current value to the previous value i.e., $\frac{y^t_{open}}{y^{t-1}_{open}}$ will be passed in the estimation of $y^t_{close}$ instead of passing $y^t_{open}$ solely.

In [ ]:
window_size = 20
batch_size = 2
lastndays = 200
epochs = 100
val_start_date = pd.Timestamp('2023-01-01', tz='Europe/Istanbul')
norm_mean_vector = tf.convert_to_tensor(np.mean(data_stock,axis=0).astype('float32'))
norm_std_vector = tf.convert_to_tensor(np.std(data_stock,axis=0).astype('float32'))
close_mean = norm_mean_vector[3]
close_std = norm_std_vector[3]

In [ ]:
val_start_date_ix = [x for x,y in enumerate(list(data_stock.index)) if y >= val_start_date][0]
open_series = data_stock['Open'].values
close_series = data_stock['Close'].values
max_close_val = max(close_series)

def convert_data(data_matrix):

  coln = data_matrix.shape[-1]
  rown = data_matrix.shape[-2]
  close_val = data_matrix[-1,3]
  open_val = data_matrix[-1,0]
  data_matrix = tf.concat([data_matrix[1:window_size+1,:4] / data_matrix[:window_size,:4],data_matrix[1:window_size+1,4:]],axis=-1)
  tfz = tf.zeros(shape=[(coln-1)],dtype='float32')
  tfz = tf.concat([data_matrix[-1,0][tf.newaxis],tfz],0)
  tfz2 = tf.zeros(shape=[(window_size-1)],dtype='float32')
  tfz2 = tf.concat([open_val[tf.newaxis],tfz2],0)
  data_matrix = (tf.concat([data_matrix[:-1,:],tfz[tf.newaxis]],0)-norm_mean_vector[None,:])/norm_std_vector[None,:]
  data_matrix = tf.concat([data_matrix,tfz2[:,tf.newaxis]],axis=-1)

  return (data_matrix,close_val)

def preprocess_data(sequence_data, data_type):

  row_sequence = sequence_data.shape[0]
  column_sequence = sequence_data.shape[1]
  sequence_data = np.array(sequence_data,dtype='float32')
  sequence_data = tf.data.Dataset.from_tensor_slices(sequence_data)
  sequence_data = sequence_data.window(size = window_size + 1, shift = 1, drop_remainder = True)
  sequence_data = sequence_data.flat_map(lambda x: x.batch(window_size+1))
  sequence_data = sequence_data.map(lambda x: convert_data(x))
  if data_type == 'training':
    sequence_data = sequence_data.shuffle(row_sequence)
  sequence_data = sequence_data.batch(batch_size).prefetch(1)
  return sequence_data

training_data = preprocess_data(data_stock[:val_start_date_ix],'training')
validation_data = preprocess_data(data_stock[(val_start_date_ix-window_size):],'validation')

for i in validation_data:
  print(f"Shape of first batch of data: {i[0].shape}")
  print(f"Shape of first batch of response data: {i[1].shape}")
  break


Shape of first batch of data: (2, 20, 8)
Shape of first batch of response data: (2,)


As shown above, each batch is composed of 2 observations and 11 days of window with all the previous information to be used. The close value of the day will be estimated using previous 10 days of information in addition to current day's open price as the rest of the information such as close, high, low are masked.

Let's see if using previous close data point in the estimation of next gives reasonable values or not and also let's use current day's open value.

In [ ]:
mape_adhoc = tf.keras.metrics.mean_absolute_percentage_error(close_series[1:val_start_date_ix],close_series[:val_start_date_ix-1]).numpy()
print(f"Mean Absolute Percentage All Data Error for Adhoc Method using Close: {mape_adhoc}")
mape_adhoc = tf.keras.metrics.mean_absolute_percentage_error(close_series[val_start_date_ix:],close_series[val_start_date_ix-1:-1]).numpy()
print(f"Mean Absolute Percentage Validation Error for Adhoc Method using Close: {mape_adhoc}")
mape_adhoc = tf.keras.metrics.mean_absolute_percentage_error(close_series[1:val_start_date_ix],open_series[1:val_start_date_ix]).numpy()
print(f"Mean Absolute Percentage All Data Error for Adhoc Method using Open: {mape_adhoc}")
mape_adhoc = tf.keras.metrics.mean_absolute_percentage_error(close_series[val_start_date_ix:],open_series[val_start_date_ix:]).numpy()
print(f"Mean Absolute Percentage Validation Error for Adhoc Method using Open: {mape_adhoc}")

Mean Absolute Percentage All Data Error for Adhoc Method using Close: 1.9531615272103435
Mean Absolute Percentage Validation Error for Adhoc Method using Close: 3.3452645056895025
Mean Absolute Percentage All Data Error for Adhoc Method using Open: 1.0780315708514465
Mean Absolute Percentage Validation Error for Adhoc Method using Open: 2.399660708500607


In both training and the validation set, the close values are less accurate in the estimation of next day's close values compared to current day's open values.

Optimizing MAPE should make more sense than any other prevalent metrics such as MSE or MAE as it mimics the return which could be obtained out of investing the derivative. We will adopt a more conservative model with a shallow depth to make the training easier as the gradient will flow through more freely when the network is shallower.



$$
y_{close}^t = y_{open}^{t}\big(1+ r(x^{t-1},x^{t-2},\ldots,x^{t-10})\big)
$$

The formula above shows how the model architecture is designed the $close$ value of the price at time $t$ will be estimated using $open$ price at time $t$ multiplied by $1 + \text{Estimated change in the value of price since the open price}$. $r$ will be estimated using a $tanh$ activation at the top layer assuming the price might go all the way down to 0 and might be doubled where both boundaries are assumed to be extreme.

`ReduceLROnPlateau` will be used in optimizing the loss function as it reduces the learning rate by multiplying the weight by a factor of $f$ which is executed at every $p$ iterations where the loss cannot be minimized further

In [ ]:

tf.keras.backend.clear_session()

def model_creation():

  inputs = Input(shape=[window_size,8])
  close_val = tf.expand_dims(inputs[:,0,7],-1)
  inputs2 = inputs[:,:,:-1]
  x = LSTM(128,input_shape=[window_size,7],kernel_initializer = tf.keras.initializers.RandomNormal(mean=0,stddev=1e-6))(inputs2)
  x = Dense(128,activation='relu',kernel_initializer = tf.keras.initializers.RandomNormal(mean=0,stddev=1e-6))(x)
  x = Dense(1,activation='tanh',kernel_initializer = tf.keras.initializers.RandomNormal(mean=0,stddev=1e-6))(x)

  y = close_val
  x  = y * (1 + x)

  model = Model(inputs=inputs,outputs=x)
  return model

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model = model_creation()
plateau_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",factor=0.8,patience=3,min_lr=1e-10)
model.compile(loss="mape",
              optimizer=optimizer,
              metrics=['mse','mae'])
history = model.fit(training_data,epochs=200,validation_data=validation_data,callbacks=[plateau_lr])

model.save(os.path.join(os.getcwd(),'stock_model.keras'))

Epoch 1/200
2914/2914 [==============================] - 44s 13ms/step - loss: 1.1163 - mse: 0.1106 - mae: 0.0460 - val_loss: 2.4050 - val_mse: 2.3482 - val_mae: 1.1821 - lr: 0.0010
Epoch 2/200
2914/2914 [==============================] - 38s 13ms/step - loss: 1.1104 - mse: 0.1108 - mae: 0.0460 - val_loss: 2.4022 - val_mse: 2.3465 - val_mae: 1.1808 - lr: 0.0010
Epoch 3/200
2914/2914 [==============================] - 39s 13ms/step - loss: 1.1102 - mse: 0.1108 - mae: 0.0461 - val_loss: 2.4173 - val_mse: 2.3560 - val_mae: 1.1878 - lr: 0.0010
Epoch 4/200
2914/2914 [==============================] - 39s 13ms/step - loss: 1.1082 - mse: 0.1105 - mae: 0.0459 - val_loss: 2.4066 - val_mse: 2.3491 - val_mae: 1.1828 - lr: 0.0010
Epoch 5/200
2914/2914 [==============================] - 38s 13ms/step - loss: 1.1072 - mse: 0.1099 - mae: 0.0460 - val_loss: 2.4200 - val_mse: 2.3579 - val_mae: 1.1891 - lr: 0.0010
Epoch 6/200
2914/2914 [==============================] - 38s 13ms/step - loss: 1.1077 - ms

The training and validation losses are quite close to the adhoc's loss although the model was iterated along $200$ epochs and the model is sufficiently complex. This might indicate that the information passed to the model is not sufficient to explain the variance in the response. The next step will be to add additional information to the input.